## Publish your Collection using the EDC / BC Geoservice

Publish you collections as WMS service. The BC geoservice provides access to publishing your collection as a WMS service using the geoDB Python client. Please refer to the [geoserver](https://docs.geoserver.org/latest/en/user/services/wms/reference.html) documentation for configuring the rest API call to your WMS/geojson service. 

You will have access to this service if you have purchased a large management account. This service is in beta mode. If you find any issues, please report them on our [GitHub Issues page](https://github.com/bcdev/xcube-hub/issues).

## Init the GeoDB client

In [3]:
from xcube_geodb.core.geodb import GeoDBClient
geodb = GeoDBClient()
geodb.whoami

'geodb_xcube-geodb-ci-user'

### Create a Database and Copy some Public Data Across

In [4]:
my_database_name = 'my-urban-eea-subset-db'

In case your database does not exist, you need to uncomment the following line and create the database. Please note, that database names must be unique, which means if another user has already used a database name you cannot create a database with the same name. 

In [5]:
#geodb.create_database(my_database_name)

<h1>Database my-urban-eea-subset-db created</h1>

Here we copy a collection from a public database, so we can publish as a geoservice. 

In [7]:
geodb.copy_collection(collection='SI001L2_LJUBLJANA_UA2018', new_collection='SI001L2_LJUBLJANA_UA2018', database='eea-urban-atlas', new_database=my_database_name)

<h1>Collection copied from eea-urban-atlas/SI001L2_LJUBLJANA_UA2018 to my-urban-eea-subset-db/SI001L2_LJUBLJANA_UA2018</h1>

You can check if the colleciton was successfully added to your database:

In [10]:
geodb.get_my_collections(database=my_database_name)

,owner,database,table_name
0,geodb_xcube-geodb-ci-user,my-urban-eea-subset-db,SI001L2_LJUBLJANA_UA2018


In [11]:
## List Collections Published on the BC WMS Service

#geodb.get_published_gs(database=my_database_name)

## Publish your Collection to the BC WMS Service

In [12]:
geodb.publish_gs(collection='SI001L2_LJUBLJANA_UA2018', database=my_database_name)

{'collection_id': 'SI001L2_LJUBLJANA_UA2018',
 'database': 'my-urban-eea-subset-db',
 'default_style': None,
 'geojson_url': 'https://stage.xcube-geodb.brockmann-consult.de/geoserver/my-urban-eea-subset-db/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=my-urban-eea-subset-db:my-urban-eea-subset-db_SI001L2_LJUBLJANA_UA2018&maxFeatures=10&outputFormat=application/json',
 'href': None,
 'name': 'SI001L2_LJUBLJANA_UA2018',
 'preview_url': 'https://stage.xcube-geodb.brockmann-consult.de/geoserver/my-urban-eea-subset-db/wms?service=WMS&version=1.1.0&request=GetMap&layers=my-urban-eea-subset-db:my-urban-eea-subset-db_SI001L2_LJUBLJANA_UA2018&bbox=4638735.5,2524979,4704862.5,2594976.25&width=690&height=768&srs=EPSG:3035&styles=&format=application/openlayers',
 'wfs_url': 'https://stage.xcube-geodb.brockmann-consult.de/geoserver/my-urban-eea-subset-db/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=my-urban-eea-subset-db%3Amy-urban-eea-subset-db_SI001L2_LJUBLJANA_UA2018&max

You can now click on the preview link which will give you an unstyled view of your WMS output.

In [13]:
## List Collections Published on the BC WMS Service

geodb.get_published_gs(database=my_database_name)

,collection_id,database,default_style,geojson_url,href,name,preview_url,wfs_url
0,SI001L2_LJUBLJANA_UA2018,my-urban-eea-subset-db,None,https://stage.xcube-geodb.brockmann-consult.de...,None,SI001L2_LJUBLJANA_UA2018,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...


### View the Data

Once you have published you collection, you can use the WMS service to visualise your collection. For this using ipyleaflet. In this example we have used a pre-defined style. You can always provide a custom style using the parameter `sld`. We have provided a class that will pass that parameter to the WMS service. Just replace WMSLayer with SldWMSLayer. The `sld`parameter is given as a URL to an SLD file.

In [14]:
url = f"http://stage.xcube-geodb.brockmann-consult.de/geoserver/{my_database_name}/wms?"


In [15]:
from ipyleaflet import Map, WMSLayer, basemaps

In [16]:
from traitlets import Unicode

class SldWMSLayer(WMSLayer):

    sld = Unicode('').tag(sync=True, o=True)


In [17]:
wmsn = WMSLayer(
    url=url,
    layers=f'{my_database_name}:{my_database_name}_SI001L2_LJUBLJANA_UA2018',
    format='image/png',
    opacity=0.5,
    attribution='Copernicus',
    styles='ljubilana'
#     sld='https://raw.githubusercontent.com/dzelge/test-gha/main/ljubilana.sld'
)

In [18]:
m = Map(basemap=basemaps.CartoDB.Positron, center=(46.0, 14.5), zoom=13)
m.add_layer(wmsn)

m

Map(center=[46.0, 14.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [19]:
geodb.unpublish_gs(collection='SI001L2_LJUBLJANA_UA2018', database=my_database_name)

True

In [ ]:
geodb.drop_collection('SI001L2_LJUBLJANA_UA2018', database=my_database_name)